In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.cluster import KMeans
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
# Define the image transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image):
    # If the image is grayscale, convert it to RGB by duplicating the single channel
    if image.mode != 'RGB':
        image = image.convert('RGB')
    return preprocess(image).unsqueeze(0)  # Add batch dimension


# Visualize the image using matplotlib
def imshow(img_tensor):

    img_tensor = img_tensor.squeeze(0)  # remove the batch dimension
    img_tensor = img_tensor.permute(1, 2, 0)  # change dimensions to (H, W, C)
    img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # un-normalize
    img_tensor = torch.clamp(img_tensor, 0, 1)  # clamp to [0,1]

    plt.imshow(img_tensor)
    plt.axis('off')
    plt.show()

In [3]:
# Load the pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()  # Set the model to evaluation mode

folder_path1 = "F:/UniOlder/Marburg_Informatik/Project_work/dataset/imagenet-classes.txt"

# Load ImageNet class labels
with open(folder_path1) as f:
    labels = [line.strip() for line in f.readlines()]


folder_path = "F:/UniOlder/Marburg_Informatik/Project_work/dataset/compressed_rotated"

root_directory = r"F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\data_set"


#folder_path = 'images/images'
file_names = os.listdir(root_directory)
file_names = [f for f in file_names if os.path.isfile(os.path.join(root_directory, f))]


# print(f"Found files: {file_names}")

E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
f

<_io.TextIOWrapper name='imagenet-classes.txt' mode='r' encoding='cp1252'>

In [8]:
# import os

# folder_path = "F:/UniOlder/Marburg_Informatik/Project_work/dataset"
# if os.path.exists(folder_path):
#     print(f"Folder exists: {folder_path}")
# else:
#     print(f"Folder does NOT exist: {folder_path}")


Folder exists: F:/UniOlder/Marburg_Informatik/Project_work/dataset


# ResNET classification - no ground truth

In [ ]:
# fig, axes = plt.subplots(5, 7, figsize=(15, 10))
# axes = axes.flatten()

# random_indexes = random.sample(range(len(file_names)), 35)
# selected_file_names = [file_names[i] for i in random_indexes]

# for idx, image_name in enumerate([file_names[i] for i in random_indexes]):

#     # Load the image
#     image_path = f'{folder_path}/{image_name}'
#     image = Image.open(image_path)

#     # Preprocess the image (handle grayscale to RGB conversion)
#     image_tensor = preprocess_image(image)

#     with torch.no_grad():  # Disable gradient calculation
#         output = model(image_tensor)

#     # Convert output probabilities to predicted class
#     _, predicted_class = output.max(1)

#     # Plot the image in the corresponding subplot
#     img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
#     img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
#     img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

#     # Convert to numpy array for plottAing
#     img_np = img_tensor.numpy()

#     axes[idx].imshow(img_np)
#     axes[idx].axis('off')  # Hide axis
#     axes[idx].set_title(labels[predicted_class.item()], fontsize=8)  # Set the title as the predicted label

# # Hide any remaining subplots if we have fewer images than subplots
# for ax in axes[len(file_names[:35]):]:
#     ax.axis('off')

# plt.tight_layout()
# plt.show()


# ResNET feature extraction and KMeans
https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34

In [4]:
def extract_features(image_tensor):
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze().numpy()


def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)


#maybe dbscan

def cluster_images(features_list, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features_list)
    return kmeans.labels_


In [5]:
# Set the amount of images to process
# AMOUNT_OF_IMAGES = 36981 # file_name.len

AMOUNT_OF_IMAGES = 6796 # file_name.len

In [6]:
# # Extract features for each image
# features_list = []

# for file_name in file_names[:AMOUNT_OF_IMAGES]:
#     try:
#         image_tensor = load_image(f'{root_directory}/{file_name}')
#         features = extract_features(image_tensor)
#         features_list.append(features)
#     except Exception as e:
#         print(f'Error processing {root_directory}/{file_name}: {e}')

In [6]:
import os
import pickle

# Folder path to save the file
# save_folder = 'NewTestOutdoor'  # Replace with the folder where you want to save the file
# save_file = 'features_list_outdoor.pkl'  # The name of the file



# # Ensure the folder exists
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder)

# # Full path to save the file
# save_path = os.path.join(root_directory, save_file)

save_path = "F:/UniOlder/Marburg_Informatik/Project_work/Outdoor/features_list_outdoor.pkl"

# # Save the features_list using pickle
# with open(save_path, 'wb') as f:
#     pickle.dump(features_list, f)

# print(f'Features list saved to {save_path}')

# Load the features_list from the saved file
with open(save_path, 'rb') as f:
    loaded_features_list = pickle.load(f)

print(f'Loaded {len(loaded_features_list)} features.')


Loaded 6796 features.


In [16]:
# features_list

# # # Folder path to save the file
# save_folder = 'emails_by_Prof_Bell'  # Replace with the folder where you want to save the file
# save_file = 'features_list_new.pkl'  # The name of the file

# # # Ensure the folder exists
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder)

# # # Full path to save the file
# save_path = os.path.join(save_folder, save_file)

# # # Save the features_list using pickle
# with open(save_path, 'wb') as f:
#     pickle.dump(features_list, f)

# print(f'Features list saved to {save_path}')



# with open(save_path, 'rb') as f:
#     features_list = pickle.load(f)

# print(f'Loaded {len(features_list)} features.')

Features list saved to emails_by_Prof_Bell\features_list_new.pkl
Loaded 36981 features.


In [7]:
## clustering section
features_array = np.array(loaded_features_list)
num_clusters = 5
clusters = cluster_images(features_array, num_clusters)

E:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
# pickle cluster labels

# with open('image_labels.pkl', 'wb') as f:
#     pickle.dump(image_labels, f)


In [11]:
file_names

['img-fm1037295(40).jpg',
 'img-fm1048507(33).jpg',
 'img-fm1056947(33).jpg',
 'img-fm1067477(4).jpg',
 'img-fm1074515(99).jpg',
 'img-fm1079108(4).jpg',
 'img-fm108(4).jpg',
 'img-fm1089589(99).jpg',
 'img-fm1090848(99).jpg',
 'img-fm1093887(99).jpg',
 'img-fm1093924(33).jpg',
 'img-fm1093925(33).jpg',
 'img-fm1093926(33).jpg',
 'img-fm1093928(33).jpg',
 'img-fm1119111(4).jpg',
 'img-fm1127939(5).jpg',
 'img-fm1130708(33).jpg',
 'img-fm1153480(99).jpg',
 'img-fm1168933(33).jpg',
 'img-fm121592(4).jpg',
 'img-fm1257815(5).jpg',
 'img-fm1257882(22).jpg',
 'img-fm1257900(40).jpg',
 'img-fm1257930(33).jpg',
 'img-fm1257931(33).jpg',
 'img-fm1257955(40).jpg',
 'img-fm1257979(33).jpg',
 'img-fm1257983(40).jpg',
 'img-fm1257984(22).jpg',
 'img-fm1290364(4).jpg',
 'img-fm1291285(99).jpg',
 'img-fm1291288(40).jpg',
 'img-fm1291383(33).jpg',
 'img-fm1291385(33).jpg',
 'img-fm1291463(40).jpg',
 'img-fm1291510(33).jpg',
 'img-fm1291575(4).jpg',
 'img-fm1292200(4).jpg',
 'img-fm1292683(4).jpg',
 '

In [8]:
import pandas as pd



# Create a DataFrame
data = {"File Name": file_names, "Cluster": clusters}
df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
output_file = "outdoor_file_clusters_100.xlsx"  # Output Excel file name
df.to_excel(output_file, index=False)

print(f"Excel file saved as: {output_file}")

Excel file saved as: outdoor_file_clusters_100.xlsx


In [ ]:
for k in range(num_clusters):
    m, n = 11, 7
    N = m*n
    fig, axes = plt.subplots(m, n, figsize=(m*2, 10))
    axes = axes.flatten()

    idx = -1
    # for idx, image_name in enumerate([file_names[i] for i in random_indexes]):
    for image_name, label in zip(file_names[:AMOUNT_OF_IMAGES], clusters):

        if label != k:
            continue

        if idx == N-1:
            break
        else:
            idx += 1

        # Load the image
        image_path = f'{folder_path}/{image_name}'
        try:
            image = Image.open(image_path)

            # Preprocess the image (handle grayscale to RGB conversion)
            image_tensor = preprocess_image(image)

            # Plot the image in the corresponding subplot
            img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
            img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
            img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

            # Convert to numpy array for plottAing
            img_np = img_tensor.numpy()

            axes[idx].imshow(img_np)
            axes[idx].axis('off')  # Hide axis
            axes[idx].set_title(label, fontsize=8)  # Set the title as the predicted label
        except Exception as e:
            print(f'Error processing {image_name}: {e}')

    # Hide any remaining subplots if we have fewer images than subplots
    for ax in axes[len(file_names[:N]):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()


In [19]:
# import os
# import pickle
# import matplotlib.pyplot as plt
# import torch
# from PIL import Image

# # Folder to save the visualizations
# output_folder = "cluster_visualizations"
# os.makedirs(output_folder, exist_ok=True)

# # Dictionary to store metadata
# visualization_metadata = {}

# # Loop through each cluster
# for k in range(num_clusters):
#     m, n = 11, 9
#     N = m * n
#     fig, axes = plt.subplots(m, n, figsize=(n*3, m*3))
#     axes = axes.flatten()

#     idx = -1
#     for image_name1, label in zip(file_names[:AMOUNT_OF_IMAGES], clusters):
#         if label != k:
#             continue

#         if idx == N - 1:
#             break
#         else:
#             idx += 1

#         # Load the image
#         image_path1 = os.path.join(folder_path, image_name1)
#         try:
#             image1 = Image.open(image_path1)

#             # Preprocess the image
#             image_tensor = preprocess_image(image1)

#             # Prepare the image for plotting
#             img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)
#             img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
#             img_tensor = torch.clamp(img_tensor, 0, 1)

#             # Convert to numpy for plotting
#             img_np = img_tensor.numpy()

#             axes[idx].imshow(img_np)
#             axes[idx].axis('off')
#             axes[idx].set_title(label, fontsize=8)
#         except Exception as e:
#             print(f"Error processing {image_name1}: {e}")

#     # Hide unused subplots
#     for ax in axes[idx + 1:]:
#         ax.axis('off')

#     plt.tight_layout()

    
#     # Save the figure
#     save_path = os.path.join(output_folder, f"cluster_{k}.png")
#     plt.savefig(save_path)
   
#     plt.close()  # Close the figure to free memory

#     # Add metadata for this cluster
#     visualization_metadata[k] = save_path

# # Save the metadata using pickle
# pickle_path = "visualization_metadata1.pkl"
# with open(pickle_path, 'wb') as f:
#     pickle.dump(visualization_metadata, f)

# print(f"Visualization metadata saved to {pickle_path}")





Visualization metadata saved to visualization_metadata1.pkl


In [9]:
import os
import random
import matplotlib.pyplot as plt
import torch
from PIL import Image

# Folder to save the visualizations
output_folder = "cluster_visualizations"
os.makedirs(output_folder, exist_ok=True)

root_directory1 = r"F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\data_set"
# Dictionary to store metadata
visualization_metadata = {}

# Set the number of images per cluster to display
images_per_cluster = 224

# Loop through each cluster
for k in range(num_clusters):
    m, n = 16, 14  # Rows and columns (16 x 14 = 224)
    N = m * n
    fig, axes = plt.subplots(m, n, figsize=(n * 2, m * 2))
    axes = axes.flatten()

    # Filter image names for the current cluster and randomize selection
    cluster_images = [(image_name, label) for image_name, label in zip(file_names, clusters) if label == k]
    random.shuffle(cluster_images)

    # Display up to `images_per_cluster` images
    for idx, (image_name1, label) in enumerate(cluster_images[:images_per_cluster]):
        # Load the image
        image_path1 = os.path.join(root_directory1, image_name1)
        try:
            image1 = Image.open(image_path1)

            # Preprocess the image
            image_tensor = preprocess_image(image1)

            # Prepare the image for plotting
            img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)
            img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
            img_tensor = torch.clamp(img_tensor, 0, 1)

            # Convert to numpy for plotting
            img_np = img_tensor.numpy()

            axes[idx].imshow(img_np)
            axes[idx].axis('off')
            axes[idx].set_title(label, fontsize=8)
        except Exception as e:
            print(f"Error processing {image_name1}: {e}")

    # Hide unused subplots
    for ax in axes[len(cluster_images[:images_per_cluster]):]:
        ax.axis('off')

    plt.tight_layout()
    # Save the figure
    save_path = os.path.join(output_folder, f"cluster_{k}.png")
    plt.savefig(save_path)
    plt.close()  # Close the figure to free memory

    # Add metadata for this cluster
    visualization_metadata[k] = save_path

print("Visualization completed and saved for all clusters.")


Visualization completed and saved for all clusters.


In [14]:
#saving model and extracted features


NameError: name 'image_name' is not defined

In [10]:
#to re-arrange the dataset into folders:

import os
import shutil


folder_path
a = 0

output_base_folder = "organized_dataset"  # Path to the output base folder
num_clusters = max(clusters) + 1  # Set to the number of clusters

# Create the output base folder if it doesn't exist
os.makedirs(output_base_folder, exist_ok=True)

# Loop through each cluster and create a corresponding folder
for cluster_id in range(num_clusters):
    cluster_folder = os.path.join(output_base_folder, str(cluster_id))
    os.makedirs(cluster_folder, exist_ok=True)

# Distribute images into corresponding cluster folders
for image_name, cluster_id in zip(file_names, clusters):
    src_path = os.path.join(root_directory1, image_name)  # Original image path
    dest_path = os.path.join(output_base_folder, str(cluster_id), image_name)  # Destination path
    try:
        shutil.copy(src_path, dest_path)  # Copy the image
#        print(f"Copied {image_name} to {dest_path}")
    except Exception as e:
        print(f"Error copying {image_name}: {e}")
    a += 1
print("Dataset organized by clusters!")


Dataset organized by clusters!


In [11]:
len_clusters = [0]*num_clusters 
for c in clusters:
    len_clusters[c] += 1 

        
len_clusters

[2191, 1083, 1212, 1104, 1206]

# till here

In [23]:
# pca = PCA(n_components=2)
# reduced_features = pca.fit_transform(features_array)
tsne = TSNE(n_components=2, random_state=42)
reduced_features = tsne.fit_transform(features_array)

KeyboardInterrupt: 

In [ ]:
def imscatter(x, y, image_paths, ax=None, zoom=0.1):
    if ax is None:
        ax = plt.gca()
    pictures = []
    for i, image_path in enumerate(image_paths):
        try:
            img = plt.imread(folder_path+'/'+image_path)
            if len(img.shape) == 2:  # grayscale
                im = OffsetImage(img, zoom=zoom, cmap='gray')
            else:  # color
                im = OffsetImage(img, zoom=zoom)
            ab = AnnotationBbox(im, (x[i], y[i]), xycoords='data', frameon=False)
            pictures.append(ax.add_artist(ab))
            ax.text(x[i], y[i] + 1.5, clusters[i], color='black', fontsize=9, ha='center')
        except Exception as e:
            print(f"Error loading image {folder_path+'/'+image_path}: {e}")
    return pictures



fig, ax = plt.subplots(figsize=(30, 20))
x_vals = reduced_features[:, 0]
y_vals = reduced_features[:, 1]

IMAGES_TO_PLOT=1500
imscatter(x_vals[:IMAGES_TO_PLOT], y_vals[:IMAGES_TO_PLOT], file_names[:IMAGES_TO_PLOT], ax=ax, zoom=0.025)
# imscatter(x_vals, y_vals, file_names[:AMOUNT_OF_IMAGES], ax=ax, zoom=0.025)

ax.set_title('Images clustering')
plt.xlim(min(x_vals)+10, max(x_vals)+10)
plt.ylim(min(y_vals)+10, max(y_vals)+10)
plt.show()

In [57]:
num_clusters

100

# TURTLE clusterisation
The code is built with the following libraries

PyTorch - 2.2.1  
torchvision - 0.17.1  
numpy  
scipy  
scikit-learn  
clip  
tqdm  
cuml - 24.02


In [ ]:
wget https://brbiclab.epfl.ch/wp-content/uploads/2024/06/turtle_tasks.zip


"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [ ]:
# pip create -n rapids-24.08 -c rapidsai -c conda-forge -c nvidia rapids=24.08 python=3.11 'cuda-version>=12.0,<=12.5'

Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© д ©«.


# Transfer learning
https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/

In [9]:
## file Rename-er

import os

def rename_files_in_folders(root_dir):
    # Iterate over all folders inside the root directory
    for folder_name in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder_name)
        
        # Check if it's a directory
        if os.path.isdir(folder_path):
            # Iterate over all files in the folder
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                
                # Check if it's a file
                if os.path.isfile(file_path):
                    # Split the file name and extension
                    name, ext = os.path.splitext(file_name)
                    
                    # Create the new file name
                    new_file_name = f"{name}({folder_name}){ext}"
                    new_file_path = os.path.join(folder_path, new_file_name)
                    
                    # Rename the file
                    os.rename(file_path, new_file_path)
                    print(f"Renamed: {file_path} -> {new_file_path}")

# Specify the root directory
root_directory = r"F:\UniOlder\Marburg_Informatik\Project_work\Outdoor"

# Call the function to rename files
rename_files_in_folders(root_directory)

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm14988.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm14988(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm1675092.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm1675092(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm812255.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm812255(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm925247.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fm925247(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fmc426906.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\img-fmc426906(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\scn-gr00090f14.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\0\scn-gr00090f14(0).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037787.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037787(12).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037788.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037788(12).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037789.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd10037789(12).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441030.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441030(12).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441031.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441031(12).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441032.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\12\img-fmd441032(12).jpg
Renamed: F:\UniO

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810455.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810455(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810456.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810456(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810460.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810460(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810467.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810467(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810475.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810475(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810476.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\33\img-fm810476(33).jpg
Renamed: F:\UniOlder\Marburg_Informatik\

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm823731.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm823731(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm823732.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm823732(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm824725.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm824725(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825270.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825270(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825277.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825277(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825279.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\40\img-fm825279(40).jpg
Renamed: F:\UniOlder\Marburg_Informatik\

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541289.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541289(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541300.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541300(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541302.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541302(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541303.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541303(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541305.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541305(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541307.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\5\img-fm1541307(5).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Projec

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082b10.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082b10(66).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f05.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f05(66).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f11.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f11(66).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f12.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f12(66).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f13.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f13(66).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f14.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\66\scn-mi02082f14(66).jpg
Renamed: F:\UniO

Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02066d09.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02066d09(83).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067b07.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067b07(83).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067b11.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067b11(83).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067c13.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067c13(83).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067e05.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067e05(83).jpg
Renamed: F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067e09.jpg -> F:\UniOlder\Marburg_Informatik\Project_work\Outdoor\83\scn-mi02067e09(83).jpg
Renamed: F:\UniO